In [1]:
import requests
import gradio as gr
import json
from getpass import getpass
from huggingface_hub import login

In [ ]:
login(token="change this into your token ") #change the token into your hf token 
API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
headers = {"Authorization": f"Bearer {'change this into your token '}"}

In [3]:
def check_model_status(url):
    try:
        response = requests.post(url, headers=headers, json={"inputs": "test"})
        return "Model is ready" if response.status_code == 200 else f"Status: {response.status_code}"
    except Exception as e:
        return f"Connection error: {str(e)}"

In [4]:
def generate_response(task, user_input):
    """Generate responses using different prompt patterns"""
    try:
        if task == "FAQ":
            prompt = f"""Answer these common questions based on examples:
            
            Q: What is your return policy?
            A: We accept returns within 30 days of purchase.
            
            Q: How do I track my order?
            A: Use the tracking number in your confirmation email.
            
            Q: {user_input}
            A: """
        elif task == "Math":
            prompt = f"""Solve this problem step-by-step. 
            Example: 
            Problem: If Jane has 3 apples and buys 5 more, how many does she have?
            Solution: 
            1. Start with initial apples: 3
            2. Add purchased apples: 3 + 5 = 8
            3. Final answer: 8
            
            Problem: {user_input}
            Solution:"""
        elif task == "Joke":
            prompt = f"""Generate a joke about {user_input} formatted as JSON with setup and punchline keys.
            
            Example:
            {{
                "setup": "Why did the programmer quit their job?",
                "punchline": "Because they didn't get arrays!"
            }}
            
            New joke:"""
        
        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 200,
                "temperature": 0.7,
                "return_full_text": False
            }
        }
        
        response = requests.post(API_URL, headers=headers, json=payload)
        output = json.loads(response.content.decode("utf-8"))
        
        if "error" in output:
            return f"Model loading error: {output['error']}"
            
        return output[0]['generated_text']
    
    except Exception as e:
        return f"Error: {str(e)}"

**Gradio**

In [5]:
with gr.Blocks() as demo:
    gr.Markdown("# Multi-Task Chatbot")
    with gr.Row():
        task_select = gr.Dropdown(["FAQ", "Math", "Joke"], label="Select Task")
        user_input = gr.Textbox(label="Your Input")
    output = gr.Textbox(label="Response", interactive=False)
    submit_btn = gr.Button("Ask")
    
    submit_btn.click(
        fn=generate_response,
        inputs=[task_select, user_input],
        outputs=output
    )

In [6]:
if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
